In [1]:
import pandas as pd

In [3]:
cases_path = 'data/cases.csv'

runners_path = 'data/runners.csv'
roles_path = 'data/roles.csv'

assistants_path = 'data/assistants.csv'

ftes_path = 'data/historical_ftes.csv'

In [4]:
df_cases = pd.read_csv(cases_path, sep = ',')

df_runners = pd.read_csv(runners_path, sep = ',')
df_roles = pd.read_csv(roles_path, sep = ',')

df_assistants = pd.read_csv(assistants_path, sep = ',')
df_ftes = pd.read_csv(ftes_path, sep = ',')

In [5]:
support_groups = ['Implementacion de Servicios Estrategicos CedEx Soporte',
                  'Operaciones CedEx Implementacion de Servicios Estrategicos CedEx Soporte Enterdev',
                  'Operaciones CedEx Soporte en Campo UNISYS']

resolved_states = ['Resuelto', 'Cerrado']

In [6]:
df_cases = (df_cases.query("Grupo in @support_groups")
                    .query("Estado in @resolved_states")
                    .query("Resumen.str.startswith('RPA')")
                    .query("Sintoma.str.startswith('ACIS')"))

df_cases['assistant_nickname'] = df_cases['Resumen'].str.split(' - ').str[0].str.upper()
df_cases['Sintoma'] = df_cases['Sintoma'].str.replace('?', '', regex = False)
df_cases['symptom_category'] = df_cases['Sintoma'].str.split(' - ').str[1]

In [7]:
df_runners = df_runners.dropna(subset = ['machine'])
df_runners['machine'] = df_runners['machine'].str.upper()
df_runners = pd.merge(df_runners[['role_id', 'machine']], df_roles[['role_id', 'assistant_id']], on = 'role_id', how = 'inner')
df_runners = df_runners.groupby(by = 'assistant_id').agg({'machine': 'nunique'}).reset_index()

In [8]:
df_assistants = (df_assistants.query("assistant_type == 'RPA'")
                                .query("assistant_state == 'Activo'"))

df_assistants['assistant_nickname'] = df_assistants['assistant_nickname'].str.upper()
df_assistants = pd.merge(df_assistants[['assistant_id', 'assistant_nickname', 'assistant_bia']], df_runners, on = 'assistant_id', how = 'inner')

In [9]:
df_ftes = (df_ftes.query("anio == 2022")
                    .query("mes in (10, 11, 12)"))

df_ftes = df_ftes.groupby(by = 'id_componente').agg({'ftes': 'mean'}).reset_index()

In [10]:
df_assistants = pd.merge(df_assistants, df_ftes, left_on = 'assistant_id', right_on = 'id_componente', how = 'inner')

In [11]:
df_cases = pd.merge(df_cases, df_assistants[['assistant_nickname', 'assistant_bia', 'machine', 'ftes']], on = 'assistant_nickname', how = 'inner')

In [12]:
df_cases

,Nro Caso,Usuario final afectado,Estado,Asignatario,Grupo,Urgencia,Sintoma,Resumen,Medio_ingreso,Fecha Apertura,Fecha Resuelto,assistant_nickname,symptom_category,assistant_bia,machine,ftes
0,40001218,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio máquina,Autoreporte,20/02/2023,20/02/2023,RPAGPC,Software,NO,20,0.541000
1,40001204,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,20/02/2023,20/02/2023,RPAGPC,Software,NO,20,0.541000
2,39992967,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,17/02/2023,17/02/2023,RPAGPC,Software,NO,20,0.541000
3,39988860,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,16/02/2023,16/02/2023,RPAGPC,Software,NO,20,0.541000
4,39974312,Eduardo Jose Freay Santamaria,Cerrado,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Conexión Remota,Autoreporte,13/02/2023,13/02/2023,RPAGPC,Software,NO,20,0.541000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,20738580,Jhon Fredy Rincon Gil,Cerrado,Julian David Loaiza Arango,Implementacion de Servicios Estrategicos CedEx...,Media,ACIS - Gestión - Pendientes usuario disponibil...,RPADIA - TCS - Revisar flujo de Transaccion,Autoreporte,7/07/2022,15/07/2022,RPADIA,Gestión,NO,7,0.000000
2964,20761817,Jehison Orlando Restrepo Valencia,Cerrado,Raul Eduardo Arango Saldarriaga,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Instalación/Reinstalación de...,RPAPUC - Reinstalación AA,Autoreporte,13/07/2022,13/07/2022,RPAPUC,Software,NO,2,0.015133
2965,20756506,Daniel Ramirez Vergara,Cerrado,Julian David Lopez Granda,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Hardware - Caída del servidor de bases ...,RPAPYP - Validar conexión,Autoreporte,12/07/2022,13/07/2022,RPAPYP,Hardware,NO,1,0.000000
2966,20740574,Daniel Ramirez Vergara,Cerrado,Raul Eduardo Arango Saldarriaga,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAPYP - Conexión Remota,Autoreporte,8/07/2022,8/07/2022,RPAPYP,Software,NO,1,0.000000


In [14]:
#tipos de urgencia
df_cases['Urgencia'].unique()

array(['Media', 'Inmediata', 'Puede Esperar', 'Alta', 'No es urgente'],
      dtype=object)

In [20]:
#se convierte la variable categórica en dummies para hacer el cálculo de los % urgencia por robot
df1 = pd.get_dummies(df_cases, columns = ['Urgencia'], drop_first = False)
df1

,Nro Caso,Usuario final afectado,Estado,Asignatario,Grupo,Sintoma,Resumen,Medio_ingreso,Fecha Apertura,Fecha Resuelto,assistant_nickname,symptom_category,assistant_bia,machine,ftes,Urgencia_Alta,Urgencia_Inmediata,Urgencia_Media,Urgencia_No es urgente,Urgencia_Puede Esperar
0,40001218,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio máquina,Autoreporte,20/02/2023,20/02/2023,RPAGPC,Software,NO,20,0.541000,0,0,1,0,0
1,40001204,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,20/02/2023,20/02/2023,RPAGPC,Software,NO,20,0.541000,0,0,1,0,0
2,39992967,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,17/02/2023,17/02/2023,RPAGPC,Software,NO,20,0.541000,0,0,1,0,0
3,39988860,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,16/02/2023,16/02/2023,RPAGPC,Software,NO,20,0.541000,0,0,1,0,0
4,39974312,Eduardo Jose Freay Santamaria,Cerrado,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Conexión Remota,Autoreporte,13/02/2023,13/02/2023,RPAGPC,Software,NO,20,0.541000,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,20738580,Jhon Fredy Rincon Gil,Cerrado,Julian David Loaiza Arango,Implementacion de Servicios Estrategicos CedEx...,ACIS - Gestión - Pendientes usuario disponibil...,RPADIA - TCS - Revisar flujo de Transaccion,Autoreporte,7/07/2022,15/07/2022,RPADIA,Gestión,NO,7,0.000000,0,0,1,0,0
2964,20761817,Jehison Orlando Restrepo Valencia,Cerrado,Raul Eduardo Arango Saldarriaga,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Instalación/Reinstalación de...,RPAPUC - Reinstalación AA,Autoreporte,13/07/2022,13/07/2022,RPAPUC,Software,NO,2,0.015133,0,0,1,0,0
2965,20756506,Daniel Ramirez Vergara,Cerrado,Julian David Lopez Granda,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Hardware - Caída del servidor de bases ...,RPAPYP - Validar conexión,Autoreporte,12/07/2022,13/07/2022,RPAPYP,Hardware,NO,1,0.000000,0,0,1,0,0
2966,20740574,Daniel Ramirez Vergara,Cerrado,Raul Eduardo Arango Saldarriaga,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAPYP - Conexión Remota,Autoreporte,8/07/2022,8/07/2022,RPAPYP,Software,NO,1,0.000000,0,0,1,0,0


In [40]:
df1.dtypes

Nro Caso                    int64
Usuario final afectado     object
Estado                     object
Asignatario                object
Grupo                      object
Sintoma                    object
Resumen                    object
Medio_ingreso              object
Fecha Apertura             object
Fecha Resuelto             object
assistant_nickname         object
symptom_category           object
assistant_bia              object
machine                     int64
ftes                      float64
Urgencia_Alta               uint8
Urgencia_Inmediata          uint8
Urgencia_Media              uint8
Urgencia_No es urgente      uint8
Urgencia_Puede Esperar      uint8
dtype: object

In [61]:
df_perc = df1.groupby('assistant_nickname').agg(
    
    perc_inmediata = (
        "Urgencia_Inmediata", 
         lambda x: round(sum(x)/len(x),3)),
    perc_puede_esperar = (
        "Urgencia_Puede Esperar", 
         lambda x: round(sum(x)/len(x),3)),
    perc_media = (
        "Urgencia_Media", 
         lambda x: round(sum(x)/len(x),3)),
    perc_alta= (
        "Urgencia_Alta", 
         lambda x: round(sum(x)/len(x),3)),
    perc_no_urg = (
        "Urgencia_No es urgente", 
         lambda x: round(sum(x)/len(x),3))
    
).reset_index()
pd.DataFrame(df_perc)

,assistant_nickname,perc_inmediata,perc_puede_esperar,perc_media,perc_alta,perc_no_urg
0,RPAAAP,0.111,0.111,0.778,0.000,0.000
1,RPAACA,0.000,0.133,0.867,0.000,0.000
2,RPAACE,0.000,0.000,1.000,0.000,0.000
3,RPAACEO,0.125,0.125,0.750,0.000,0.000
4,RPAACG,0.000,0.000,1.000,0.000,0.000
...,...,...,...,...,...,...
190,RPAVPD,0.100,0.100,0.800,0.000,0.000
191,RPAVPM,0.240,0.120,0.560,0.040,0.040
192,RPAVSA,0.130,0.000,0.826,0.043,0.000
193,RPAVUL,0.119,0.071,0.810,0.000,0.000


In [69]:
df_final = pd.merge(df1,df_perc,on='assistant_nickname', how='left')
display(df_final.head(),
        df_final.shape)

,Nro Caso,Usuario final afectado,Estado,Asignatario,Grupo,Sintoma,Resumen,Medio_ingreso,Fecha Apertura,Fecha Resuelto,...,Urgencia_Alta,Urgencia_Inmediata,Urgencia_Media,Urgencia_No es urgente,Urgencia_Puede Esperar,perc_inmediata,perc_puede_esperar,perc_media,perc_alta,perc_no_urg
0,40001218,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio máquina,Autoreporte,20/02/2023,20/02/2023,...,0,0,1,0,0,0.018,0.053,0.93,0.0,0.0
1,40001204,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,20/02/2023,20/02/2023,...,0,0,1,0,0,0.018,0.053,0.93,0.0,0.0
2,39992967,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,17/02/2023,17/02/2023,...,0,0,1,0,0,0.018,0.053,0.93,0.0,0.0
3,39988860,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,16/02/2023,16/02/2023,...,0,0,1,0,0,0.018,0.053,0.93,0.0,0.0
4,39974312,Eduardo Jose Freay Santamaria,Cerrado,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,ACIS - Software - Conexión remota y reinicio,RPAGPC - Conexión Remota,Autoreporte,13/02/2023,13/02/2023,...,0,0,1,0,0,0.018,0.053,0.93,0.0,0.0


(2968, 25)